# Experiment 008: Use Better Baseline (70.647306)

Verify and submit the better baseline from snapshot 21165874980.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
import json

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

print("Functions defined")

Functions defined


In [2]:
# Load the better baseline
better_file = '/home/nonroot/snapshots/santa-2025/21165874980/code/submission_candidates/candidate_001.csv'
df = pd.read_csv(better_file)
print(f"Loaded {len(df)} rows")
print(df.head())

Loaded 20100 rows
      id                     x                      y                  deg
0  001_0  s-48.196086194214246    s58.770984615214225                s45.0
1  002_0   s0.1540970696213643  s-0.03854074269478543  s203.62937773065684
2  002_1  s-0.1540970696213643   s-0.5614592573052146  s23.629377730656792
3  003_0    s1.123655816140301     s0.781101815992563    s111.125132292893
4  003_1     s1.23405569584216     s1.275999500663759     s66.370622269343


In [3]:
# Calculate total score
def calculate_score(df):
    total_score = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            print(f"Warning: N={n} has {len(n_trees)} trees")
            continue
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        total_score += side**2 / n
    return total_score

total_score = calculate_score(df)
print(f"Total score: {total_score:.6f}")

Total score: 70.647306


In [4]:
# Check for overlaps
def check_overlaps(df):
    overlaps_found = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        polygons = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            polygons.append(poly)
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    inter = polygons[i].intersection(polygons[j])
                    if inter.area > 1e-10:
                        overlaps_found.append((n, i, j, inter.area))
    return overlaps_found

print("Checking for overlaps...")
overlaps = check_overlaps(df)
if overlaps:
    print(f"Found {len(overlaps)} overlaps!")
    for n, i, j, area in overlaps[:10]:
        print(f"  N={n}: trees {i} and {j} overlap with area {area:.10f}")
else:
    print("No overlaps found!")

Checking for overlaps...


No overlaps found!


In [ ]:
# Copy to submission
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(better_file, '/home/submission/submission.csv')
print(f"Copied to /home/submission/submission.csv")

# Save metrics
os.makedirs('/home/code/experiments/008_better_baseline', exist_ok=True)
metrics = {'cv_score': total_score}
with open('/home/code/experiments/008_better_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")